In [2]:
import sqlite3
from tkinter import *
from tkinter import messagebox

# Connect or create database
conn = sqlite3.connect("contacts.db")
cursor = conn.cursor()
cursor.execute("""
    CREATE TABLE IF NOT EXISTS contacts (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        phone TEXT NOT NULL,
        email TEXT,
        address TEXT
    )
""")
conn.commit()

# Functions
def add_contact():
    name = name_entry.get()
    phone = phone_entry.get()
    email = email_entry.get()
    address = address_entry.get()

    if name and phone:
        cursor.execute("INSERT INTO contacts (name, phone, email, address) VALUES (?, ?, ?, ?)",
                       (name, phone, email, address))
        conn.commit()
        messagebox.showinfo("Success", "Contact added successfully!")
        clear_entries()
        view_contacts()
    else:
        messagebox.showwarning("Input Error", "Name and Phone are required.")

def view_contacts():
    listbox.delete(0, END)
    cursor.execute("SELECT id, name, phone FROM contacts")
    for row in cursor.fetchall():
        listbox.insert(END, f"{row[0]} | {row[1]} | {row[2]}")

def search_contact():
    listbox.delete(0, END)
    query = search_entry.get()
    cursor.execute("SELECT id, name, phone FROM contacts WHERE name LIKE ? OR phone LIKE ?", 
                   (f"%{query}%", f"%{query}%"))
    for row in cursor.fetchall():
        listbox.insert(END, f"{row[0]} | {row[1]} | {row[2]}")

def delete_contact():
    try:
        selected = listbox.get(listbox.curselection())
        contact_id = selected.split('|')[0].strip()
        cursor.execute("DELETE FROM contacts WHERE id=?", (contact_id,))
        conn.commit()
        messagebox.showinfo("Deleted", "Contact deleted successfully.")
        view_contacts()
    except:
        messagebox.showwarning("Error", "Select a contact to delete.")

def update_contact():
    try:
        selected = listbox.get(listbox.curselection())
        contact_id = selected.split('|')[0].strip()
        name = name_entry.get()
        phone = phone_entry.get()
        email = email_entry.get()
        address = address_entry.get()

        if name and phone:
            cursor.execute("""
                UPDATE contacts
                SET name=?, phone=?, email=?, address=?
                WHERE id=?
            """, (name, phone, email, address, contact_id))
            conn.commit()
            messagebox.showinfo("Updated", "Contact updated successfully.")
            clear_entries()
            view_contacts()
        else:
            messagebox.showwarning("Input Error", "Name and Phone are required.")
    except:
        messagebox.showwarning("Error", "Select a contact to update.")

def fill_details(event):
    try:
        selected = listbox.get(listbox.curselection())
        contact_id = selected.split('|')[0].strip()
        cursor.execute("SELECT name, phone, email, address FROM contacts WHERE id=?", (contact_id,))
        row = cursor.fetchone()
        clear_entries()
        name_entry.insert(0, row[0])
        phone_entry.insert(0, row[1])
        email_entry.insert(0, row[2])
        address_entry.insert(0, row[3])
    except:
        pass

def clear_entries():
    name_entry.delete(0, END)
    phone_entry.delete(0, END)
    email_entry.delete(0, END)
    address_entry.delete(0, END)

# GUI setup
root = Tk()
root.title("Contact Book")
root.geometry("700x500")
root.config(bg="#F0F8FF")

# Form
Label(root, text="Name:", bg="#F0F8FF").place(x=20, y=20)
name_entry = Entry(root, width=30)
name_entry.place(x=120, y=20)

Label(root, text="Phone:", bg="#F0F8FF").place(x=20, y=60)
phone_entry = Entry(root, width=30)
phone_entry.place(x=120, y=60)

Label(root, text="Email:", bg="#F0F8FF").place(x=20, y=100)
email_entry = Entry(root, width=30)
email_entry.place(x=120, y=100)

Label(root, text="Address:", bg="#F0F8FF").place(x=20, y=140)
address_entry = Entry(root, width=30)
address_entry.place(x=120, y=140)

# Buttons
Button(root, text="Add", width=12, bg="#32CD32", command=add_contact).place(x=400, y=20)
Button(root, text="Update", width=12, bg="#1E90FF", command=update_contact).place(x=400, y=60)
Button(root, text="Delete", width=12, bg="#FF4500", command=delete_contact).place(x=400, y=100)

# Search
Label(root, text="Search by Name or Phone:", bg="#F0F8FF").place(x=20, y=190)
search_entry = Entry(root, width=30)
search_entry.place(x=200, y=190)
Button(root, text="Search", width=10, command=search_contact).place(x=450, y=187)

# Contact list
listbox = Listbox(root, width=80, height=12, font=('Courier', 10))
listbox.place(x=20, y=230)
listbox.bind('<<ListboxSelect>>', fill_details)

# View All
Button(root, text="View All Contacts", width=20, bg="#DAA520", command=view_contacts).place(x=250, y=430)

# Initial view
view_contacts()

root.mainloop()
